In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
from deltalake.writer import write_deltalake
from deltalake import DeltaTable
import os

In [3]:
pd.set_option('display.width', 78)
pd.set_option('display.max_columns',6)

In [4]:
os.makedirs("data/temps_lake", exist_ok=True)

In [5]:
landtemps = pd.read_csv('data/landtempssample.csv',
    names=['stationid','year','month','avgtemp','latitude',
      'longitude','elevation','station','countryid','country'],
    skiprows=1,
    parse_dates=[['month','year']])

C:\Users\user\AppData\Local\Temp\ipykernel_432\3925585316.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  landtemps = pd.read_csv('data/landtempssample.csv',
C:\Users\user\AppData\Local\Temp\ipykernel_432\3925585316.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  landtemps = pd.read_csv('data/landtempssample.csv',


In [6]:
# Read the CSV without parsing dates initially
landtemps = pd.read_csv(
    'data/landtempssample.csv',
    names=['stationid', 'year', 'month', 'avgtemp', 'latitude',
           'longitude', 'elevation', 'station', 'countryid', 'country'],
    skiprows=1
)

# Combine 'month' and 'year' into a single datetime column
landtemps['date'] = pd.to_datetime(
    landtemps['year'].astype(str) + '-' + landtemps['month'].astype(str),
    format='%Y-%m',
    errors='coerce'
)

# Drop the original 'month' and 'year' columns if they're no longer needed
landtemps.drop(columns=['month', 'year'], inplace=True)

# Display the first few rows to verify
print(landtemps.head())


     stationid  avgtemp  latitude  ...  countryid        country       date
0  USS0010K01S     5.27   39.9000  ...         US  United States 2000-04-01
1  CI000085406    18.04  -18.3500  ...         CI          Chile 1940-05-01
2  USC00036376     6.22   34.3703  ...         US  United States 2013-12-01
3  ASN00024002    22.93  -34.2833  ...         AS      Australia 1963-02-01
4  ASN00028007      NaN  -14.7803  ...         AS      Australia 2001-11-01

[5 rows x 9 columns]


In [7]:
landtemps.shape

(100000, 9)

In [21]:
print(landtemps.dtypes)

stationid            object
avgtemp             float64
latitude            float64
longitude           float64
elevation           float64
station              object
countryid            object
country              object
date         datetime64[ns]
dtype: object


In [22]:
write_deltalake("data/temps_lake", landtemps)

In [23]:
tempsdelta = DeltaTable("data/temps_lake", version=0)
type(tempsdelta)

deltalake.table.DeltaTable

In [24]:
tempsdfv1 = tempsdelta.to_pandas()
tempsdfv1.shape

(100000, 9)

In [25]:
write_deltalake("data/temps_lake", landtemps.head(1000), mode="overwrite")

In [26]:
tempsdfv2 = DeltaTable("data/temps_lake", version=1).to_pandas()
tempsdfv2.shape

(1000, 9)

In [27]:
write_deltalake("data/temps_lake", landtemps.head(1000), mode="append")

In [28]:
tempsdfv3 = DeltaTable("data/temps_lake", version=2).to_pandas()
tempsdfv3.shape

(2000, 9)

In [29]:
DeltaTable("data/temps_lake", version=0).to_pandas().shape

(100000, 9)